# Test macro for Bokeh client visualization
* NOT FOR STANDARD USERS
* This notebook to be used for developpers to test feature of bokeh interactive  - standalone plotting

In [ ]:
from bokeh.io import output_notebook, show
#, output_file
from bokeh.plotting import figure
from bokeh.layouts import column, row
from bokeh.models import CustomJS, ColumnDataSource, Slider
from RootInteractive.InteractiveDrawing.bokeh.bokehTools import *
from RootInteractive.InteractiveDrawing.bokeh.bokehDraw import *
from bokeh.io import output_notebook
output_notebook()


In [ ]:
#init test random data
df = pd.DataFrame(np.random.randint(0, 100, size=(100, 4)), columns=list('ABCD'))
df["Category"]=df.eval("A%5")
df.head(10)
initMetadata(df)
df.meta.metaData = {'A.AxisTitle': "A (cm)", 'B.AxisTitle': "B (cm/s)", 'C.AxisTitle': "C (s)", 'D.AxisTitle': "D (a.u.)"}
#print(df.metaData)

In [ ]:
figureArray = [
    [['A'], ['C-A'], {"color": "red", "size": 7, "colorZvar":"C"}],
    [['A'], ['C+A', 'C-A']],
    [['B'], ['C+B', 'C-B'],{"color": "red", "size": 7, "colorZvar":"C"}],
    [['D'], ['sin(D/10)', 'sin(D/20)*0.5', 'sin(D/40)*0.25'], {"size": 10}],
    ['table']
]

figureLayout: str = '((0,1,2),(3, x_visible=1),commonX=1,x_visible=1,y_visible=0,plot_height=250,plot_width=1000)'
tooltips = [("VarA", "(@A)"), ("VarB", "(@B)"), ("VarC", "(@C)"), ("VarD", "(@D)")]
pAll,layoutList,source,data  =bokehDrawArray(df, "A>0", figureArray, layout=figureLayout, color="blue", size=4, tooltips=tooltips)
show(pAll)

In [ ]:
logging.disable(1)

In [ ]:
output_file("test2.html")

def test_FigureArray():
    """
    Test of the bokehDrawArray to draw figure array -
    :return:
    """
    # 1.) create figures
    figureLayout: str = '((0,1,2, plot_height=200),(3, x_visible=1),commonX=1,x_visible=1,y_visible=0,plot_height=150,plot_width=1000)'
    tooltips = [("VarA", "(@A)"), ("VarB", "(@B)"), ("VarC", "(@C)"), ("VarD", "(@D)")]
    #dummy,source,figureList  = bokehDrawArray(df, "A>0", figureArray, layout=figureLayout, color="blue", size=4, tooltips=tooltips)
    dummy,source,figureList,data  = bokehDrawArray(df, "A>0", figureArray, layout=figureLayout, color="blue", size=4, tooltips=tooltips)

    # 2.) make CDS
    cdsOrig = ColumnDataSource(df.query("A>0"))
    cdsSel =  source
    widgetDict={"cdsOrig":cdsOrig, "cdsSel":cdsSel}
    sliders=[]
    # 3.) create sliders 
    for a in df.columns:
        length=df[a].max()-df[a].min()
        sliderRange= RangeSlider(start=df[a].min(), end=df[a].max(), value=(df[a].min(), df[a].max()), step=length*0.01, title=a+"Range")
        sliders.append([sliderRange])
        widgetDict[a+"Range"]=sliderRange
    # make callback
    mycallback=makeJScallback(widgetDict)
    display(mycallback)
    for a in sliders:
        figureList.append(a)
        a[0].js_on_event('value',mycallback)
        a[0].js_on_change('value',mycallback)
        
    sliderRange2= RangeSlider(start=0.1, end=4, value=(0.1,4) , step=0.1, title="Range2")
    sliderRange2.js_on_change('value', mycallback)
    figureList.append([sliderRange2])
    pAll=    gridplotRow(figureList)
    show(pAll)
    return pAll
    #print(widgetDict)
    
figure=test_FigureArray() 